<h1>Separating JSON Data into Columns (Movies)</h1>

In [3]:
import csv, json, gensim, datetime, time
from gensim import corpora

rowslist = []
genreslist = []
#productioncompanieslist = []
#productioncountrieslist = []

def readJSON(row, rownumber, list, keyword):
    jsonstring = row[rownumber]

    jsonobj = json.loads(jsonstring)

    for jsonelement in jsonobj:
        keywordelement = jsonelement[keyword]       
        list.append(keywordelement.lower()) 
    
    return list
    
#read file
with open('tmdb_10000_movies.csv', encoding='utf-8') as csv_file: #change accordingly
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        try:        
            if line_count == 0:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            else: #if line_count != 5:
                eachnewrow = []
                genres = []
                productioncompanies = []
                productioncountries = []

                eachnewrow.append(row[0]) #budget

                #handle json genres
                genres = readJSON(row, 1, genres, 'name')

                eachnewrow.append(genres) #genres
                genreslist.append(genres)

                #row[2] homepage removed
                eachnewrow.append(row[3]) #id

                #row[4] original_language removed
                #row[5] original_title removed  
                #row[6] overview removed                    
                eachnewrow.append(row[7]) #popularity                  

                #handle json productioncompanies
                productioncompanies = readJSON(row, 8, productioncompanies, 'name')

                eachnewrow.append(len(productioncompanies)) #productioncompanies
                #productioncompanieslist.append(productioncompanies)                  

                #handle json productioncountries
                productioncountries = readJSON(row, 9, productioncountries, 'name')

                eachnewrow.append(len(productioncountries)) #productioncountries
                #productioncountrieslist.append(productioncountries) 

                weekday = ""
                day = ""
                month = ""
                year = ""                 

                #handle releasedate 
                if row[10] != "":
                    d = datetime.datetime.strptime(row[10] ,'%Y-%m-%d')

                    weekdayword = d.strftime('%a')
                    weekday = time.strptime(weekdayword, '%a').tm_wday + 1

                    day = d.strftime('%d')

                    monthword = d.strftime('%b')
                    month = time.strptime(monthword,'%b').tm_mon

                    year = d.strftime('%Y')

                eachnewrow.append(weekday) #weekday
                eachnewrow.append(day) #day 
                eachnewrow.append(month) #month
                eachnewrow.append(year) #year

                eachnewrow.append(row[11]) #revenue
                eachnewrow.append(row[12]) #runtime
                #row[13] spoken_languages removed  
                #row[14] status removed 
                #row[15] tagline removed   
                eachnewrow.append(row[16]) #title
                eachnewrow.append(row[17]) #vote_average
                eachnewrow.append(row[18]) #vote_count                  

                WR = (float(row[18]) / (float(row[18])+25000)) * float(row[17]) + (25000 / (float(row[18]) + 25000)) * 7.0            

                eachnewrow.append(WR) #weighted_rating

                rowslist.append(eachnewrow)

                line_count += 1
        except Exception as e:
            print(f'Row: {line_count} has Exception')    
            line_count += 1          
    print(f'Processed {line_count} lines.')   
                  
genresdictionary = corpora.Dictionary(genreslist)
token_to_id_genres = genresdictionary.token2id                

#productioncompaniesdictionary = corpora.Dictionary(productioncompanieslist)
#token_to_id_productioncompanies = productioncompaniesdictionary.token2id
                  
#productioncountriesdictionary = corpora.Dictionary(productioncountrieslist)
#token_to_id_productioncountries = productioncountriesdictionary.token2id   

def assigningValues(vector, listofzeros):
    for counter, value in enumerate(vector): 
        index = 0       
        for value2 in listofzeros:                      
            if index == value[0]:
                listofzeros[index] = value[1]    
            index += 1
                  
    return listofzeros
                  
#write to file
with open('processed_tmdb_10000_movies.csv', mode='w', newline='', encoding='utf-8') as movies_file: #change accordingly
    writer2 = csv.writer(movies_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    writer2.writerow(['budget'] + list(genresdictionary.values()) + ['id', 'popularity', 'production_companies', 'production_countries'] 
        #+ list(productioncompaniesdictionary.values()) + list(productioncountriesdictionary.values()) 
        + ['weekday', 'day', 'month', 'year', 'revenue', 'runtime', 'title', 'vote_average', 'vote_count', 'weighted_rating'])              
    
    for eachrow in rowslist:
        listofzerosgenres = [0] * len(token_to_id_genres)
        #listofzerosproductioncompanies = [0] * len(token_to_id_productioncompanies)
        #listofzerosproductioncountries = [0] * len(token_to_id_productioncountries)                   
                  
        vec_genres = genresdictionary.doc2bow(eachrow[1])
        #vec_productioncompanies = productioncompaniesdictionary.doc2bow(eachrow[4])
        #vec_productioncountries = productioncountriesdictionary.doc2bow(eachrow[5])                   

        listofzerosgenres = assigningValues(vec_genres, listofzerosgenres)
        #listofzerosproductioncompanies = assigningValues(vec_productioncompanies, listofzerosproductioncompanies)
        #listofzerosproductioncountries = assigningValues(vec_productioncountries, listofzerosproductioncountries)

        writer2.writerow(eachrow[0:1] + listofzerosgenres + eachrow[2:6] 
            #+ listofzerosproductioncompanies + listofzerosproductioncountries 
            + eachrow[6:16])
                  
    print('\nOutput Success!')                 

Column names are budget, genres, homepage, id, original_language, original_title, overview, popularity, production_companies, production_countries, release_date, revenue, runtime, spoken_languages, status, tagline, title, vote_average, vote_count
Processed 6 lines.

Output Success!


<h1>Separating JSON Data into Columns (Credits)</h1>

In [4]:
import csv, json, gensim
from gensim import corpora

rowslist = []
departmentslist = []
jobtitleslist = []

#read file
with open('tmdb_10000_credits.csv', encoding='utf-8') as csv_file: #change accordingly
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        try:
            if line_count == 0:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            else: #if line_count != 5:
                eachnewrow = []
                departments = []
                jobtitles = []

                malecastsnum = 0
                femalecastsnum = 0
                unknowngendercastsnum = 0    
                totalcasts = 0

                malecrewsnum = 0
                femalecrewsnum = 0
                unknowngendercrewsnum = 0
                totalcrews = 0

                eachnewrow.append(row[0]) #id

                #handle json casts
                jsonstring = row[1]

                jsonobj = json.loads(jsonstring)

                for jsonelement in jsonobj:
                    gender = jsonelement['gender']       
                    if gender == 0:
                      unknowngendercastsnum += 1
                    elif gender == 1:
                      femalecastsnum += 1
                    elif gender == 2:
                      malecastsnum += 1

                    totalcasts += 1

                eachnewrow.append(malecastsnum) #malecastsnum
                eachnewrow.append(femalecastsnum) #femalecastsnum                  
                eachnewrow.append(unknowngendercastsnum) #unknowngendercastsnum 
                eachnewrow.append(totalcasts) #totalcasts                   

                #handle json crews              
                jsonstring2 = row[2]

                jsonobj2 = json.loads(jsonstring2)

                for jsonelement in jsonobj2:
                    gender = jsonelement['gender']       
                    if gender == 0:
                      unknowngendercrewsnum += 1
                    elif gender == 1:
                      femalecrewsnum += 1
                    elif gender == 2:
                      malecrewsnum += 1  

                    totalcrews += 1

                    department = jsonelement['department']
                    departments.append(department.lower())         

                    jobtitle = jsonelement['job']
                    jobtitles.append(jobtitle.lower())                    

                eachnewrow.append(malecrewsnum) #malecastsnum
                eachnewrow.append(femalecrewsnum) #femalecastsnum                  
                eachnewrow.append(unknowngendercrewsnum) #unknowngendercastsnum                   
                eachnewrow.append(totalcrews) #totalcrews  

                eachnewrow.append(departments) #departments
                departmentslist.append(departments)  

                eachnewrow.append(jobtitles) #jobtitles
                jobtitleslist.append(jobtitles)

                rowslist.append(eachnewrow)

                line_count += 1
        except Exception as e:
            print(f'Row: {line_count} has Exception')    
            line_count += 1   

    print(f'Processed {line_count} lines.')
                  
departmentsdictionary = corpora.Dictionary(departmentslist)
token_to_id_departments = departmentsdictionary.token2id
#print(f'Dictionary for Departments: {token_to_id_departments}') 
#print(f'\nNo. of Departments: {len(token_to_id_departments)}')
                  
jobtitlesdictionary = corpora.Dictionary(jobtitleslist)
token_to_id_jobtitles = jobtitlesdictionary.token2id
#print(f'Dictionary for Job Titles: {token_to_id_jobtitles}')        
#print(f'No. of Job Titles: {len(token_to_id_jobtitles)}')                   

def assigningValues(vector, listofzeros):
    for counter, value in enumerate(vector): 
        index = 0       
        for value2 in listofzeros:                      
            if index == value[0]:
                listofzeros[index] = value[1]    
            index += 1
                  
    return listofzeros                  
                  
#write to file
with open('processed_tmdb_10000_credits.csv', mode='w', newline='', encoding='utf-8') as credits_file: #change accordingly
    writer = csv.writer(credits_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    writer.writerow(['id', 'malecastsnum', 'femalecastsnum', 'unknowngendercastsnum', 'totalcasts', 'malecrewsnum', 'femalecrewsnum', 'unknowngendercrewsnum', 'totalcrews'] 
        + list(departmentsdictionary.values()) + list(jobtitlesdictionary.values()))              
    
    for eachrow in rowslist:
        listofzerosdepartments = [0] * len(token_to_id_departments)
        listofzerosjobtitles = [0] * len(token_to_id_jobtitles)                         
                  
        vec_departments = departmentsdictionary.doc2bow(eachrow[9])
        vec_jobtitles = jobtitlesdictionary.doc2bow(eachrow[10])   

        listofzerosdepartments = assigningValues(vec_departments, listofzerosdepartments)
        listofzerosjobtitles = assigningValues(vec_jobtitles, listofzerosjobtitles)

        writer.writerow(eachrow[0:9] + listofzerosdepartments + listofzerosjobtitles)

    print('\nOutput Success!')
                  

Column names are id, cast, crew
Processed 6 lines.

Output Success!


<h1>Merging Two Files</h1>

In [93]:
import pandas as pd

movies = pd.read_csv("processed_tmdb_10000_movies.csv") #change accordingly
credits = pd.read_csv("processed_tmdb_10000_credits.csv") #change accordingly
merged = pd.merge(movies, credits, left_on=['id'],
                   right_on=['id'], 
                   how='left')
merged.to_csv("processed_tmdb_data.csv", index=False)

print('\nOutput Success!')  


Output Success!


<h1>Append New Processed Data to Previous Merged File</h1>

In [36]:
import csv

with open('main_processed_tmdb_data.csv', mode='a', newline='', encoding='utf-8') as main_file:
    writer3 = csv.writer(main_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    with open('processed_tmdb_data.csv', mode='r', newline='') as new_file:
        csv_reader = csv.reader(new_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                line_count += 1
            else:
                writer3.writerow(row)
                line_count += 1
            
    print(f'Processed {line_count} lines.')
    
    print('\nOutput Success!')  

Processed 4804 lines.

Output Success!
